In [1]:
import numpy as np
import time
import mujoco
import mujoco.viewer

In [ ]:
modelfilename = 'test4n.xml'
m = mujoco.MjModel.from_xml_path(modelfilename)
d = mujoco.MjData(m)
mrec = [[] for _ in range(45)]
print(m.dof_armature)
print(len(m.dof_armature))
sinamp = 0.1
# m.dof_armature[6:]=[10]*14

class isi_controller():
    def __init__(self,m):
        self.fs = 0
        self.control_vel = np.zeros(m.nu)

    def controller(self,m,d):
        # dofd = [10]*14
        # m.dof_armature[6:] = dofd
        rls = 0
        lls = 0
        
        # mujoco.mj_forward(m, d)
        rfid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link3')
        lfid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link6')
        rhid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link1') #Composite_HIPRY
        lhid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link4') #Composite_HIPLY
        rfpos = d.xpos[rfid].copy()
        lfpos = d.xpos[lfid].copy()
        rhip  = d.xpos[rhid].copy()
        lhip  = d.xpos[lhid].copy()
        vecr2l = lhip - rhip
        vecr2lxynorm = np.sqrt(vecr2l[0]*vecr2l[0] + vecr2l[1]*vecr2l[1])
        fRy = np.array([vecr2l[0]/vecr2lxynorm,vecr2l[1]/vecr2lxynorm,0])
        fRz = np.array([0,0,1])
        fRx = np.cross(fRy,fRz)
        fRT = np.array([fRx,fRy,fRz])
        fp = vecr2l/2 + rhip
        fpi = -fRT@fp
        
        h = 1
        g = 9.8
        omega = np.sqrt(g/h)
        mujoco.mj_subtreeVel(m,d)
        vcom = d.subtree_linvel[1].copy()
        com  = d.subtree_com[1].copy()
        eta = com + vcom/omega
        frfpos = fRT@rfpos + fpi
        flfpos = fRT@lfpos + fpi
        fcom = fRT@com + fpi
        fvcom = fRT@vcom
        fvp = fRT@(vcom/omega)
        feta = fcom + fvp
        
        if d.sensordata[0]<0.01 and d.sensordata[1]<0.01 and d.sensordata[2]<0.01 and d.sensordata[3]<0.01:
            rls = 1
        if d.sensordata[4]<0.01 and d.sensordata[5]<0.01 and d.sensordata[6]<0.01 and d.sensordata[7]<0.01:
            lls = 1
        
        if feta[1]-frfpos[1] > 0.2 and self.fs == 0: #eta[1] > 0.08 #0.07
            pass
            ###########################################
            print('time:',d.time,'fs_state_transition:0->1')
            self.fs = 1
        if flfpos[1]-feta[1] > 0.2 and self.fs == 1: #eta[1] < -0.08
            pass
            ###########################################
            print('time:',d.time,'fs_state_transition:1->0')
            self.fs = 0
            
        kp = 80
        kd = 19
        kp2 = kp
        kd2 = kd
        kp3 = 5
        kd3 = 3
        initald = 5
        #参数说明：关节名称    Kp  位置目标  Kd     速度目标 D
        cl = {  'HIPRY'  :[   kp,     0,  kd,     0,  initald],
                'HIPRX'  :[   kp,     0,  kd,     0,  initald],
                'HIPRZ'  :[   kp,     0,  kd,     0,  initald],
                'KNEERY' :[   kp,     0,  kd,     0,  initald],
                'ANKLERY':[   kp2,    0,  kd2,    0,  initald],
                'ANKLERX':[   kp3,    0,  kd3,    0,  initald],
                'HIPLY'  :[   kp,     0,  kd,     0,  initald],
                'HIPLX'  :[   kp,     0,  kd,     0,  initald],
                'HIPLZ'  :[   kp,     0,  kd,     0,  initald],
                'KNEELY' :[   kp,     0,  kd,     0,  initald],
                'ANKLELY':[   kp2,    0,  kd2,    0,  initald],
                'ANKLELX':[   kp3,    0,  kd3,    0,  initald],
                'WAISTY' :[   kp,     0,  kd,     0,  initald],
                'WAISTX' :[   kp,     0,  kd,     0,  initald]}
            
        #knee:0.7 ank:0.12 
        output = 0
        hipyor = 0
        hipyol = 0
        apitr = 0
        apitl = 0
        kneer = 0
        kneel = 0
        hipzr = 0
        hipzl = 0
        kneeliftangle = 0.7 #0.7 #1.5 0.7
        amoo = 0.12 #0.08
        swingang = 0.1
        anklebias = 0.05 #0.08 $0.16 #0.06
            
        cl['HIPRX'][1]  = output
        cl['HIPLX'][1]  = output
        cl['HIPRY'][1]  = hipyor
        cl['HIPLY'][1]  = hipyol
        cl['KNEERY'][1] = kneer
        cl['KNEELY'][1] = kneel
        cl['ANKLERY'][1]= apitr
        cl['ANKLELY'][1]= apitl
        cl['ANKLERY'][4]= 5
        cl['ANKLELY'][4]= 5
        cl['ANKLERX'][4]= 0
        cl['ANKLELX'][4]= 0
        
        for jn,tm in cl.items():
            m.dof_armature[m.jnt(jn).id+5] = tm[4]
            # self.control_vel[m.jnt(jn).id-1] += -tm[0]*(d.qpos[m.jnt(jn).id+6]-tm[1])
            # d.ctrl[m.jnt(jn).id-1] = (-tm[2]*(d.qvel[m.jnt(jn).id+5]-self.control_vel[m.jnt(jn).id-1]))*tm[4]
            d.ctrl[m.jnt(jn).id-1] = (-tm[0]*(d.qpos[m.jnt(jn).id+6]-tm[1]) -tm[2]*(d.qvel[m.jnt(jn).id+5]-tm[3]))*tm[4]

def set_cam_viewer(m, viewer):
    # 设置镜头跟随
    viewer.cam.type = mujoco.mjtCamera.mjCAMERA_TRACKING
    viewer.cam.trackbodyid = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_BODY, "link0")
    viewer.cam.distance = 3.0# 设置摄像机距离 
    viewer.cam.elevation = -20# 设置摄像机仰角
    
    # 设置摄像机为固定点模式
    # viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
    # 设定摄像机的位置和朝向
    # viewer.cam.lookat = [0, 0, 0]  # 摄像机视线的目标点
    # viewer.cam.distance = 5.0      # 摄像机与目标点的距离
    # viewer.cam.elevation = -20     # 摄像机仰角
    # viewer.cam.azimuth = 45        # 摄像机方位角

mycontroller1 = isi_controller(m)
with mujoco.viewer.launch_passive(m, d) as viewer:
  # set_cam_viewer(m, viewer)
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  ii = 0
#   qst = pin.Quaternion(pin.AngleAxis(0,np.array([0,0,1])).matrix()).coeffs().copy()
#   qslt = qst[3]
#   qst[3] = qst[2]
#   qst[2] = qst[1]
#   qst[1] = qst[0]
#   qst[0] = qslt
#   d.qpos[3:7] = qst
  mujoco.mj_forward(m, d)
  while viewer.is_running() and time.time() - start < 25:
    step_start = time.time()

    # if ii % 2 == 0:
    #     mycontroller1.controller(m,d)
    # ii += 1

    mrec[24].append(d.ctrl[0])
    mrec[25].append(d.ctrl[3])
    mrec[26].append(d.ctrl[4])
    
    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics. 
    mujoco.mj_step(m, d)
    mrec[1].append(d.subtree_com[1][0])
    mrec[2].append(d.subtree_com[1][2])
    mrec[3].append(d.subtree_com[1][1])
    mrec[4].append(sinamp*np.sin(d.time - 2.5)) #这里也要跟着改
    angmomm = np.zeros((3,m.nv))
    mujoco.mj_angmomMat(m,d,angmomm,0)
    hhere = angmomm@d.qvel
    mrec[5].append(hhere[0])
    mrec[6].append(hhere[1])
    mrec[7].append(hhere[2])
    # mrecord8.append(d.qpos[8])
    # mrecord9.append(d.qpos[14])
    mujoco.mj_subtreeVel(m,d)
    mrec[30].append(d.subtree_linvel[1][0])
    mrec[31].append(d.subtree_linvel[1][1])
    mrec[32].append(d.subtree_linvel[1][2])

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
20


In [ ]:
modelfilename = 'test4n.xml'
m = mujoco.MjModel.from_xml_path(modelfilename)
d = mujoco.MjData(m)
mujoco.mj_forward(m, d)

M = np.zeros((m.nv, m.nv))
mujoco.mj_fullM(m, M, d.qM)
Mbj = M[:6,6:12]
Mjj = M[6:12,6:12]
jacp = np.zeros((3,m.nv))
jacr = np.zeros((3,m.nv))
mujoco.mj_jacBody(m,d,jacp,jacr,mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_BODY, 'link3'))
jaclf = np.block([[jacp],[jacr]])
jaclfb = jaclf[:,:6]
jaclfj = jaclf[:,6:12]
print('Mbj:',Mbj)
print('Mjj:',Mjj)
print('jacp:',jacp)
print('jacr:',jacr)
print('jaclf:',jaclf)
print('jaclfb:',jaclfb)
print('jaclfj:',jaclfj)
inv_jaclfbT = np.linalg.inv(jaclfb.T)
Mexpect = jaclfj.T@inv_jaclfbT@Mbj
print('Mexpect',Mexpect)

Mbj: [[-3.35000000e+00  0.00000000e+00 -1.11022302e-16 -8.50000000e-01
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  3.35000000e+00  3.00000000e-02  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.00000000e-02  1.11022302e-16  0.00000000e+00 -3.00000000e-02
  -3.00000000e-02  0.00000000e+00]
 [ 4.50000000e-03  2.59768333e+00  3.30000000e-02  4.50000000e-03
   4.50000000e-03  2.83333333e-04]
 [ 2.60203333e+00 -1.54914841e-19  1.08440388e-16  8.00833333e-01
   4.63333333e-03  0.00000000e+00]
 [-5.02500000e-01  3.00000000e-02  9.68333333e-03 -1.27500000e-01
   0.00000000e+00  0.00000000e+00]]
Mjj: [[ 2.26703333e+00 -1.54914841e-19  9.73381582e-17  7.15833333e-01
   4.63333333e-03  0.00000000e+00]
 [-1.54914841e-19  2.26268333e+00  3.00000000e-02  0.00000000e+00
   0.00000000e+00  2.83333333e-04]
 [ 9.73381582e-17  3.00000000e-02  9.68333333e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.15833333e-01  0.00000000e+00  0.00000000e+00  2.90833333e-01
   4.63333333e